In [2]:
import numpy as np
import os 
import math 
import matplotlib.pyplot as plt 
import rasterio as rio
from rasterio.plot import show 

# Save Image As Plots 

In [8]:
%%time
def calculate_ndvi(red_band, nir_band):
    """Calculate NDVI."""
    return (nir_band - red_band) / (nir_band + red_band)

def plot_ndvi(ndvi, output_path):
    """Plot NDVI and save the plot."""
    plt.figure(figsize=(8, 8))
    plt.imshow(ndvi, cmap='gist_earth', vmin=1, vmax=-1)  # Adjust colormap as needed
    plt.colorbar(label='NDVI')
    plt.title('NDVI Plot')
    plt.xlabel('Column')
    plt.ylabel('Row')
    plt.savefig(output_path)
    plt.close()

def process_sentinel_images(input_folder, output_folder):
    """Process Sentinel images in the input folder."""
    # Create output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all image files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.img'):
            # Construct full path to the image file
            img_path = os.path.join(input_folder, file_name)

            # Read all bands
            with rio.open(img_path) as src:
                red_band = src.read(1).astype(np.float32)
#                 blue_band = src.read(2).astype(np.float32)
#                 green_band = src.read(3).astype(np.float32)
                nir_band = src.read(4).astype(np.float32)

            # Calculate NDVI
            ndvi = calculate_ndvi(red_band, nir_band)

            # Plot and save NDVI
            output_path = os.path.join(output_folder, f'ndvi_{os.path.splitext(file_name)[0]}.png')
            plot_ndvi(ndvi, output_path)

# Define input and output folders
input_folder = 'F:\\Project\\layerstack\\Image_Iter\\Input_Raster'  # Update with your input folder path
output_folder = 'F:\\Project\\layerstack\\Image_Iter\\Output_NDVI'  # Update with your output folder path

# Process Sentinel images
process_sentinel_images(input_folder, output_folder)


CPU times: total: 17 s
Wall time: 24.5 s


# Save Output As .tif

In [14]:
def calculate_ndvi(red_band, nir_band):
    """Calculate NDVI."""
    return (nir_band - red_band) / (nir_band + red_band)

def plot_ndvi(ndvi, output_path, crs, transform):
    """Plot NDVI and save the plot."""
    with rio.open(output_path, 'w', driver='GTiff', 
                  height=ndvi.shape[0], width=ndvi.shape[1],
                  count=1, dtype=np.float32, crs=crs, transform=transform) as dst:
        dst.write(ndvi, 1)

def process_sentinel_images(input_folder, output_folder):
    """Process Sentinel images in the input folder."""
    # Create output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all image files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.img'):
            # Construct full path to the image file
            img_path = os.path.join(input_folder, file_name)

            # Read all bands
            with rio.open(img_path) as src:
                red_band = src.read(1).astype(np.float32)
                nir_band = src.read(4).astype(np.float32)
                crs = src.crs
                transform = src.transform

            # Calculate NDVI
            ndvi = calculate_ndvi(red_band, nir_band)

            # Plot and save NDVI
            output_path = os.path.join(output_folder, f'ndvi_{os.path.splitext(file_name)[0]}.tif')
            plot_ndvi(ndvi, output_path, crs, transform)

# Process Sentinel images
process_sentinel_images(input_folder, output_folder)
